In [2]:
from sklearn.datasets import load_wine
wine_data=load_wine()

In [4]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [6]:
print(wine_data['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [7]:
feat_data=wine_data['data']

In [8]:
labels=wine_data['target']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test=train_test_split(feat_data,labels,test_size=0.3,
                                              random_state=101)

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler=MinMaxScaler()

In [13]:
scaled_x_train=scaler.fit_transform(x_train)
scaled_x_test=scaler.fit_transform(x_test)

In [14]:
import tensorflow as tf
from tensorflow import estimator

C:\Users\codehax\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
x_train.shape

(124, 13)

In [16]:
feat_cols=[tf.feature_column.numeric_column('x', shape=[13])]

In [19]:
#classs= 3 cause 3 types of alcohol
deep_model=estimator.DNNClassifier(hidden_units=[13,13,13],
                                  feature_columns=feat_cols,
                                  n_classes=3,
                                  optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\codehax\\AppData\\Local\\Temp\\tmp9jm8muqr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020157E45F60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
input_fn=estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},
                                        y=y_train,
                                        shuffle=True,
                                        batch_size=10,
                                        num_epochs=5)

In [21]:
deep_model.train(input_fn=input_fn,steps=500)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\codehax\AppData\Local\Temp\tmp9jm8muqr\model.ckpt.
INFO:tensorflow:loss = 11.074572, step = 1
INFO:tensorflow:Saving checkpoints for 62 into C:\Users\codehax\AppData\Local\Temp\tmp9jm8muqr\model.ckpt.
INFO:tensorflow:Loss for final step: 7.1108847.


In [22]:
input_fn_eval=estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},
                                             shuffle=False)

In [24]:
preds=list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\codehax\AppData\Local\Temp\tmp9jm8muqr\model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [27]:
predictions=[p['class_ids'][0] for p in preds]

In [28]:
from sklearn.metrics import confusion_matrix,classification_report

In [31]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.53      0.69        19
           1       0.71      1.00      0.83        22
           2       1.00      1.00      1.00        13

   micro avg       0.83      0.83      0.83        54
   macro avg       0.90      0.84      0.84        54
weighted avg       0.88      0.83      0.82        54



# Keras

In [32]:
from tensorflow.contrib.keras import models

In [33]:
dnn_keras_model=models.Sequential()

In [34]:
from tensorflow.contrib.keras import layers
dnn_keras_model.add(layers.Dense(units=13,input_dim=13,activation='relu'))

In [35]:
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))
dnn_keras_model.add(layers.Dense(units=3,activation='softmax'))

In [36]:
from tensorflow.contrib.keras import losses,optimizers,metrics,activations

In [37]:
dnn_keras_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

In [38]:
dnn_keras_model.fit(scaled_x_train,y_train,epochs=50)

Epoch 1/50
124/124 [==============================] - 1s 6ms/step - loss: 1.0923 - acc: 0.3952
Epoch 2/50
124/124 [==============================] - 0s 104us/step - loss: 1.0842 - acc: 0.3952
Epoch 3/50
124/124 [==============================] - 0s 88us/step - loss: 1.0767 - acc: 0.3952
Epoch 4/50
124/124 [==============================] - 0s 97us/step - loss: 1.0688 - acc: 0.3952
Epoch 5/50
124/124 [==============================] - 0s 97us/step - loss: 1.0615 - acc: 0.3952
Epoch 6/50
124/124 [==============================] - 0s 65us/step - loss: 1.0538 - acc: 0.3952
Epoch 7/50
124/124 [==============================] - 0s 65us/step - loss: 1.0459 - acc: 0.3952
Epoch 8/50
124/124 [==============================] - 0s 97us/step - loss: 1.0371 - acc: 0.3952
Epoch 9/50
124/124 [==============================] - 0s 97us/step - loss: 1.0279 - acc: 0.3952
Epoch 10/50
124/124 [==============================] - 0s 97us/step - loss: 1.0180 - acc: 0.3952
Epoch 11/50
124/124 [==================

In [42]:
predictions=dnn_keras_model.predict_classes(scaled_x_test)

In [43]:
from sklearn.metrics import classification_report
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92        18
           1       0.91      0.91      0.91        22
           2       1.00      0.93      0.96        14

   micro avg       0.93      0.93      0.93        54
   macro avg       0.93      0.93      0.93        54
weighted avg       0.93      0.93      0.93        54



# Layers API

In [44]:
import pandas as pd

In [51]:
onehot_y_train=pd.get_dummies(y_train).as_matrix()

C:\Users\codehax\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [46]:
onehot_y_train=pd.get_dummies(y_test).as_matrix()

C:\Users\codehax\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [52]:
num_feat=13
num_hidden1=13
num_hidden2=13
num_outputs=3
learning_rate=0.01

In [54]:
from tensorflow.contrib.layers import fully_connected

In [55]:
x=tf.placeholder(tf.float32,shape=[None,num_feat])

In [56]:
y_true=tf.placeholder(tf.float32,shape=[None,3])

In [57]:
actf=tf.nn.relu

In [58]:
hidden1=fully_connected(x,num_hidden1,activation_fn=actf)
hidden2=fully_connected(hidden1,num_hidden2,activation_fn=actf)
output=fully_connected(hidden2,num_outputs)

In [60]:
loss=tf.losses.softmax_cross_entropy(onehot_labels=y_true,logits=output)

In [71]:
optimizer=tf.train.AdamOptimizer(learning_rate)
train=optimizer.minimize(loss)
init=tf.global_variables_initializer()
training_steps=1000

In [72]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(training_steps):
        sess.run(train,feed_dict={x:scaled_x_train,y_true:onehot_y_train})
    logits=output.eval(feed_dict={x:scaled_x_test})
    preds=tf.argmax(logits,axis=1)
    results=preds.eval()

In [73]:
print(classification_report(results,y_test))

              precision    recall  f1-score   support

           0       1.00      0.63      0.78        30
           1       1.00      0.92      0.96        24
           2       0.00      0.00      0.00         0

   micro avg       0.76      0.76      0.76        54
   macro avg       0.67      0.52      0.58        54
weighted avg       1.00      0.76      0.86        54



C:\Users\codehax\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
